In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re

In [3]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [4]:
train_df.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
64068,667488407518,Hand in Glove \n\nWasn't Hand in Glove on thei...,0,0,0,0,0,0
54561,569492804188,"""\n\n New image problems \n\nYou again uploade...",0,0,0,0,0,0
30748,320053612537,"Vankenta here: Well, it seems that there is no...",0,0,0,0,0,0
55105,575253713169,"Welcome!\n\nHello, Dparisi, and welcome to Wik...",0,0,0,0,0,0
35076,364961921992,"""\nTry right-clicking on the LoC link then Sav...",0,0,0,0,0,0


In [5]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [6]:
train_df.describe()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,9.585100e+04,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000
mean,4.994359e+11,0.096368,0.010068,0.053301,0.003182,0.049713,0.008492
std,2.890136e+11,0.295097,0.099832,0.224635,0.056320,0.217352,0.091762
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.001297e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.501088e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.999882e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# fill NaN with string "unknown"
test_df.fillna('unknown',inplace=True)

In [8]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in train is {}'.format(len(train_df)))
print('Total rows in test is {}'.format(len(test_df)))

Total rows in train is 95851
Total rows in test is 226998


In [9]:
print(train_df[cols_target].sum())

obscene          5109
insult           4765
toxic            9237
severe_toxic      965
identity_hate     814
threat            305
dtype: int64


In [10]:
X = train_df.comment_text
test_X = test_df.comment_text

In [11]:
print(X.shape, test_X.shape)

(95851,) (226998,)


In [12]:
# import and instantiate CountVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=(1,4), max_features=50000, min_df=2, lowercase=False, max_df = 0.8)
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=0.8, max_features=50000, min_df=2,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
# learn the vocabulary in the training data, then use it to create a document-term matrix
X_dtm = vect.fit_transform(X)
# examine the document-term matrix created from X_train
X_dtm

<95851x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 7792863 stored elements in Compressed Sparse Row format>

In [14]:
# transform the test data using the earlier fitted vocabulary, into a document-term matrix
test_X_dtm = vect.transform(test_X)
# examine the document-term matrix from X_test
test_X_dtm

<226998x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 20943598 stored elements in Compressed Sparse Row format>

In [15]:
def find_most_important_features(cv, model):
    # supported only for linear kernels
    feature_names = cv.get_feature_names()
    coefs_with_fns = sorted(zip(model.coef_[0], feature_names))
    n = 50
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n+1):-1])
    
    print('\nMost informative features\n')
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print(coef_1, fn_1, '   ', coef_2, fn_2)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
logreg = LogisticRegression(C=9.0)

# create submission file
submission_binary = pd.read_csv('../input/sample_submission.csv')

for label in cols_target:
    print('\n... Processing {}'.format(label))
    y = train_df[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print('Training Precision-Recall-F1 is {}'.format(precision_recall_fscore_support(y, y_pred_X)))
    print('\nConfusion matrix:\n', confusion_matrix(y, y_pred_X))
    find_most_important_features(vect, logreg)
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_binary[label] = test_y_prob


... Processing obscene
Training accuracy is 0.9899948878989264
Training Precision-Recall-F1 is (array([ 0.99120263,  0.96524664]), array([ 0.99829186,  0.84263065]), array([ 0.99473462,  0.89978054]), array([90742,  5109]))

Confusion matrix:
 [[90587   155]
 [  804  4305]]

Most informative features

-6.36923446514 Thanks     25.3469709592 fuck
-5.01140868676 source was     25.0080463556 fucking
-4.41685583193 YOU HAVE     20.1620232472 Fuck
-4.31691909352 Ah     17.755287282 bitch
-3.74678975253 quality     17.1507077014 asshole
-3.6531170254 be reverted     16.0106876349 bullshit
-3.62954390205 your parents     15.9163614385 ass
-3.62483114981 Talk     15.775288359 FUCK
-3.59253919103 source     15.4646175279 dick
-3.5565457055 that am     15.3022602951 FUCKING
-3.53995112325 example     15.1843999501 shit
-3.46557848666 could     15.0351021318 cunt
-3.46010784625 ve     12.4384519196 suck
-3.45564911008 your computer     12.2076032187 fucked
-3.35273756967 Go away     11.692625631

In [17]:
submission_binary.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.008019,0.001851,0.006784,0.000722,0.005412,0.001986
1,6102620,0.005995,0.000269,0.001480,0.000274,0.000349,0.000826
2,14563293,0.000369,0.000117,0.000413,0.000070,0.000479,0.000112
3,21086297,0.004815,0.004390,0.010236,0.000813,0.004350,0.000507
4,22982444,0.000244,0.000053,0.000289,0.000106,0.000010,0.000142


In [18]:
# generate submission file
submission_binary.to_csv('submission_binary.csv',index=False)